In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
# Given distribution values
values = [1, np.cbrt(4) ** 2, np.cbrt(6) ** 2, np.cbrt(4) ** 2, 1]

# Corresponding d values (-2, -1, 0, 1, 2)
d_values = [-2, -1, 0, 1, 2]

# Calculate normalization constant T (sum of values)
T = sum(values)

# Normalize the values to get the probability distribution
prob_distribution = [x / T for x in values]

# Calculate the expectation of norm (|d|)
expectation_vl_norm_eta_2 = sum(prob * abs(d) * abs(d) for prob, d in zip(prob_distribution, d_values))
print(expectation_vl_norm_eta_2)

# Output the probability distribution and expectation
for i, prob in enumerate(prob_distribution, start=-2):
    print(f"P(d_{i}) = {prob:.4f}")
    

1.2608948097255643
P(d_-2) = 0.0967
P(d_-1) = 0.2437
P(d_0) = 0.3193
P(d_1) = 0.2437
P(d_2) = 0.0967


In [3]:
# Given distribution values
values = [1, np.cbrt(6) ** 2, np.cbrt(15) ** 2, np.cbrt(20) ** 2, np.cbrt(15) ** 2, np.cbrt(6) ** 2, 1]
# values = [1, np.cbrt(6), np.cbrt(15), np.cbrt(20), np.cbrt(15), np.cbrt(6), 1]

d_values = [-3, -2, -1, 0, 1, 2, 3]

# Calculate normalization constant T (sum of values)
T = sum(values)

# Normalize the values to get the probability distribution
prob_distribution = [x / T for x in values]

# Calculate the expectation of norm (|d|)
expectation_vl_norm_eta_3 = sum(prob * abs(d) * abs(d) for prob, d in zip(prob_distribution, d_values))
print(expectation_vl_norm_eta_3)

# Output the probability distribution and expectation
for i, prob in enumerate(prob_distribution, start=-3):
    print(f"P(d_{i}) = {prob:.4f}")

2.010917525591277
P(d_-3) = 0.0355
P(d_-2) = 0.1174
P(d_-1) = 0.2162
P(d_0) = 0.2619
P(d_1) = 0.2162
P(d_2) = 0.1174
P(d_3) = 0.0355


In [4]:
def calculate_delta(b):
    delta = (((math.pi * b)**(1 / b) * b) / (2 * math.pi * math.e))**(1 / (2 * (b - 1)))
    # print(f"b, delta: {b}, {delta}")
    return delta

def calculate_reduction_runtime_log2(b, d, r):
    n = d-r
    delta = calculate_delta(b)
    rho_log2 = math.log2(math.ceil((n**2 / b**2) * math.log2(n))) # need to make it precise in the case b does not divide others
    T_svp_log2 = 0.265*b+16.4 #2**(0.265*b+16.4)
    runtime_log2 = rho_log2 + math.log2(n) + T_svp_log2
    return runtime_log2, delta

In [5]:
from scipy.special import beta
from scipy.integrate import quad
import Q_Toffoli_cost as QTC

def compute_probability_log2(R, v_norm, d, r):
    """
    Compute the probability 'p' based on the given formula.
    
    Parameters:
        R (list or array): List of R_i values
        v_norm (float): Norm of vector ||v_l||
        d (int): Dimension parameter
        r (int): guessing number
        
    Returns:
        float: Approximation of the probability 'p'
    """
    def integrand(t, power):
        """Integrand function: (1 - t^2)^power"""
        return (1 - t**2)**power
    
    # Initialize probability product
    p = 1.0
    p_log2 = 0
    beta_value = beta((d - r - 1) / 2, 1 / 2)  # Precompute Beta function

    for i in range(d - r):
        r_i = R[i] / (2 * v_norm)
        max_limit = max(-r_i, -1)
        power = (d - r - 3) / 2
        
        integral_value, _ = quad(integrand, -1, max_limit, args=(power))

        term = 1 - (2 / beta_value) * integral_value
        term_log2 = math.log2(term)

        p *= term
        p_log2 += term_log2
    
    return p_log2

def calculate_k(b, delta, d):
    """
    Calculate the value of k based on the formula:
    k = min(floor(sqrt(b / log_q_delta)), d)
    """
    # print(math.log(delta, q))
    return min(math.floor(math.sqrt(b / math.log(delta, q))), d)

def calculate_R_i(i, d, k, q, delta, b):
    """
    Calculate the value of R_i based on the given conditions:
    R_i = q, if i <= d - k
          delta^(-2 * (i - (d - k) - 1) + k) * q^(k - b / k), otherwise
    """
    if i <= d - k:
        return q
    else:
        exponent = -2 * (i - (d - k) - 1) + k
        # print(exponent)

        log2_delta = math.log2(delta)
        log2_q = math.log2(q)

        # Calculate log2 of the value
        log2_value = exponent * log2_delta + ((k - b) / k) * log2_q
        
        return 2**log2_value
    
def calculate_quantum_depth_count_log2(n, r, log2_L_j_list):
    depth_const, cost_const = QTC.compute_Toffoli_depth_and_cost(n, r, log2_L_j_list)
    # print(f"Depth const: {depth_const}, Cost const: {cost_const}")
    if n == 512:
        log2_depth = 1.302 * r + depth_const
        log2_count = 1.302 * r + cost_const
    elif n == 768:
        log2_depth = 1.108 * r + depth_const
        log2_count = 1.108 * r + cost_const
    elif n == 1024:
        log2_depth = 1.108 * r + depth_const
        log2_count = 1.108 * r + cost_const
    else:
        raise ValueError("Invalid value for n. Supported values are 512, 768, 1024.")
    
    return log2_depth, log2_count

def log2_sum_stable(x_prime, y_prime):
    max_prime = max(x_prime, y_prime)
    min_prime = min(x_prime, y_prime)
    result = max_prime + math.log2(1 + 2**(min_prime - max_prime))
    return result



In [6]:
import os
import datetime

# Ensure the calculate and utility functions are implemented
# calculate_reduction_runtime_log2, calculate_k, calculate_R_i,
# compute_probability_log2, calculate_quantum_depth_log2,
# log2_sum_stable

q = 3329  # Replace with actual value
n_list = [512, 768, 1024]
vl_expect_list = [expectation_vl_norm_eta_3, expectation_vl_norm_eta_2, expectation_vl_norm_eta_2]

# Create a directory to store the results that is with today's date
output_dir = f"result_with_cost{datetime.datetime.now().strftime('%Y-%m-%d')}"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

for idx in range(3):
    n = n_list[idx]
    vl_expect_each = vl_expect_list[idx]
    d = 2 * n  # -1 because the last 1 entry is skipped

    # all_result_fn = os.path.join(output_dir, f"all_cost_log2_results_{n}.csv")
    best_result_fn = os.path.join(output_dir, f"best_cost_log2_results_{n}.csv")

    # Load existing results if files exist
    # if os.path.exists(all_result_fn):
    #     all_results_df = pd.read_csv(all_result_fn)
    # else:
    #     all_results_df = pd.DataFrame()

    if os.path.exists(best_result_fn):
        best_results_df = pd.read_csv(best_result_fn)
    else:
        best_results_df = pd.DataFrame()

    previous_cost = float('inf')  # Track previous cost
    increase_count = 0  # Track number of consecutive increases in cost

    for r in range(1, n + 1):
        # Skip if r is already processed
        if not best_results_df.empty and r in best_results_df['r'].values:
            continue

        vl_norm = np.sqrt(vl_expect_each * (d - r))
        d_ab = 2 * n - r
        a = n
        b = n - r

        best_cost = float('inf')  # Initialize to a very high value
        best_details = None  # To store the best cost details

        for block_size in range(90, d-r+1): # b needs to be bigger than 90 to apply BDGL sieve cost estimation. In addition, blocksize was chosen to be bigger than 90 expreimentally.
            T_red_log2, delta = calculate_reduction_runtime_log2(block_size, d, r)

            if delta <= 1:
                continue

            k = calculate_k(b, delta, d_ab)

            if k >= d_ab or k <= b:
                continue

            R_values = [calculate_R_i(i, d_ab, k, q, delta, b) for i in range(1, d_ab + 1)]
            prob_log2 = compute_probability_log2(R_values, vl_norm, d, r)
            log2_L_j_list = [math.log2(R_values[j - 1]) for j in range(1, d_ab + 1)]


            T_hyb_log2, count_hyp_log2 = calculate_quantum_depth_count_log2(n, r, log2_L_j_list)

            cost_log2_hyp = T_hyb_log2 - prob_log2
            cost_log2_red = T_red_log2 - prob_log2

            total_cost_log2 = log2_sum_stable(cost_log2_hyp, cost_log2_red)

            # Save current result
            current_result = {
                "n": n,
                "r": r,
                "block_size": block_size,
                "total_cost_log2": total_cost_log2,
                "T_hyb_log2": T_hyb_log2,
                "T_red_log2": T_red_log2,
                "prob_log2": prob_log2,
                "delta": delta,
                "k": k,
                "Toffoli_cost_log2_hyp": cost_log2_hyp,
                "R_values": R_values
            }
            # all_results_df = pd.concat([all_results_df, pd.DataFrame([current_result])], ignore_index=True)

            # Check if this is the best cost so far
            if total_cost_log2 < best_cost:
                best_cost = total_cost_log2
                best_details = current_result

        # Save the best result for this r
        if best_details:
            best_results_df = pd.concat([best_results_df, pd.DataFrame([best_details])], ignore_index=True)

        # # Check if cost increased
        # if best_cost > previous_cost:
        #     increase_count += 1
        # else:
        #     increase_count = 0

        # Update previous cost
        previous_cost = best_cost

        # Stop if cost has increased for 5 consecutive iterations
        # if increase_count >= 5:
        #     print(f"Stopping early for n={n} after r={r} due to consecutive cost increases.")
        #     break

        # Save results to CSV after each r
        # all_results_df.to_csv(all_result_fn, index=False)
        best_results_df.to_csv(best_result_fn, index=False)

        # print(f"Results for r={r} saved to {all_result_fn} and {best_result_fn}")
        print(f"Results for r={r} saved to {best_result_fn}")

print("All computations completed.")


Results for r=1 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=2 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=3 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=4 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=5 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=6 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=7 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=8 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=9 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=10 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=11 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
Results for r=12 saved to result_with_cost2025-07-03\best_cost_log2_results_512.csv
R